Extract the data from the csv

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

raw_csv_data = np.loadtxt("/content/Audiobooks_data.csv", delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

Balance the data

In [ ]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
      zero_targets_counter +=1
      if zero_targets_counter > num_one_targets:
        indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

Standardize the inputs

In [ ]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

Shuffle the data

In [ ]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

Split the dataset into train, validation and test

In [ ]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1786.0 3579 0.4990220732048058
222.0 447 0.4966442953020134
229.0 448 0.5111607142857143


Save the three datasets in *.npz

In [ ]:
np.savez('/content/Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('/content/Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('/content/Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [ ]:
Data

In [ ]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

Model

In [ ]:
input_size = 10
output_size = 2
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks = [early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 2
)

Epoch 1/100
36/36 - 2s - loss: 0.5337 - accuracy: 0.7279 - val_loss: 0.4579 - val_accuracy: 0.7830 - 2s/epoch - 50ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4079 - accuracy: 0.7837 - val_loss: 0.4011 - val_accuracy: 0.7897 - 171ms/epoch - 5ms/step
Epoch 3/100
36/36 - 0s - loss: 0.3718 - accuracy: 0.8030 - val_loss: 0.3879 - val_accuracy: 0.8054 - 146ms/epoch - 4ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3575 - accuracy: 0.8083 - val_loss: 0.3790 - val_accuracy: 0.7875 - 141ms/epoch - 4ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3500 - accuracy: 0.8125 - val_loss: 0.3661 - val_accuracy: 0.8121 - 164ms/epoch - 5ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3427 - accuracy: 0.8156 - val_loss: 0.3615 - val_accuracy: 0.8009 - 158ms/epoch - 4ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3373 - accuracy: 0.8192 - val_loss: 0.3644 - val_accuracy: 0.7942 - 154ms/epoch - 4ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3371 - accuracy: 0.8145 - val_loss: 0.3656 - val_accuracy: 0.8098 - 167ms/epoch - 5ms/step


Test the model

In [ ]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.33. Test accuracy: 79.69%
